# TwelveLabs Multimodal Embedding Search on Amazon Bedrock
Work with TwelveLabs Marengo Embed 2.7 Model

![TwelveLabs Embedding](./12labs-embed.png)

The TwelveLabs Marengo Embed 2.7 model generates embeddings from video, text, audio, or image inputs. These embeddings can be used for similarity search, clustering, and other machine learning tasks. The model supports asynchronous inference through the StartAsyncInvoke API.

In this sample, we demonstrate how to use the TwelveLabs Marengo Embed 2.7 model, available through Amazon Bedrock, to generate embeddings for a sample video and perform dynamic search.

In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install faiss-cpu==1.7.4

In [ ]:
import boto3
import json

bedrock = boto3.client('bedrock-runtime')
s3 = boto3.client('s3')

In [ ]:
model_id = 'twelvelabs.marengo-embed-2-7-v1:0'

s3_bucket = '<YOUR_S3_BUCKET>'
s3_prefix = '<YOUR_S3_PREFIX>' # For example: 'twelvelabs'
aws_account_id = '<YOUR AWS ACCOUNT ID>'

## Download a Sample Video and Upload to S3 as Input
We'll use the TwelveLabs Marengo model to generate embeddings from this video and perform content-based search.

In [ ]:
# Download a sample video to local disk
sample_name = 'NetflixMeridian.mp4'
source_url = f'https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/335119c4-e170-43ad-b55c-76fa6bc33719/NetflixMeridian.mp4'
!curl {source_url} --output {sample_name}

# Upload to S3
s3_input_key = f'{s3_prefix}/video/{sample_name}'
s3.upload_file(sample_name, s3_bucket, s3_input_key)
print(f"Uploaded to s3://{s3_bucket}/{s3_input_key}")

## Generate Multimodal Embeddings Using TwelveLabs Marengo 2.7 Model
We use Bedrock’s [StartAsyncInvoke](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_StartAsyncInvoke.html) to run the embedding task asynchronously. In this example, the video is hosted on S3—ideal for handling large video files. The API also supports providing the video as a base64-encoded string within the payload. Refer to the [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-marengo.html?trk=769a1a2b-8c19-4976-9c45-b6b1226c7d20&sc_channel=el) for more details.

In [ ]:
import uuid

s3_output_prefix = f'{s3_prefix}/output/{uuid.uuid4()}'
response = bedrock.start_async_invoke(
    modelId=model_id,
    modelInput={
        "inputType": "video",
        "mediaSource": {
            "s3Location": {
                "uri": f's3://{s3_bucket}/{s3_input_key}',
                "bucketOwner": aws_account_id
            }
        }
    },
    outputDataConfig={
        "s3OutputDataConfig": {
            "s3Uri": f's3://{s3_bucket}/{s3_output_prefix}'
        }
    }
)

# Print Job ID
invocation_arn = response["invocationArn"]
print("Async Job Started")
print("Invocation Arn:", invocation_arn)

The result will be available in S3 once the task is complete. The code snippet below wait until the output.json file is ready and read it from the output path specified in your request.

In [ ]:
import time
from IPython.display import clear_output
from datetime import datetime

def wait_for_output_file(s3_bucket, s3_prefix, invocation_arn):
    # Wait until task complete
    status = None
    while status not in ["Completed", "Failed", "Expired"]:
        response = bedrock.get_async_invoke(invocationArn=invocation_arn)
        status = response['status']
        clear_output(wait=True)
        print(f"Embedding task status: {status}")
        time.sleep(5)

    # List objects in the prefix
    response = s3.list_objects_v2(Bucket=s3_bucket, Prefix=f'{s3_prefix}')

    # Look for output.json
    data = []
    output_key = None
    for obj in response.get('Contents', []):
        if obj['Key'].endswith('output.json'):
            output_key = obj['Key']
            if output_key:
                obj = s3.get_object(Bucket=s3_bucket, Key=output_key)
                content = obj['Body'].read().decode('utf-8')
                data += json.loads(content).get("data")

    return data

In [ ]:
from IPython.display import display, JSON
output = wait_for_output_file(s3_bucket, s3_output_prefix, invocation_arn)
display(JSON(output))

## Store the Embeddings in a Vector Database
In this example, we use [FAISS](https://faiss.ai/index.html), an open-source in-memory vector database, to store the embeddings generated in the previous steps to serve light search as an example.
For production applications, a stateful and scalable solution such as [Amazon OpenSearch Service](https://aws.amazon.com/opensearch-service/) or [Amazon S3 Vector](https://aws.amazon.com/s3/features/vectors/) is recommended.

Create a index in FAISS:

In [ ]:
import faiss
import numpy as np

# Create an index for cosine similarity (IndexFlatIP = inner product)
embedding_dim = 1024
index = faiss.IndexFlatIP(embedding_dim)

for data in output:
    embedding = np.array([float(d) for d in data["embedding"]], dtype=np.float32)
    embedding = embedding.reshape(1, -1)
    index.add(embedding)  # Add the embedding to the index

To perform a similarity search in a vector database, you must generate the query embedding using the same model that was used to generate the stored embeddings.

In this example, we perform a simple text-based search by invoking the Marengo model with the following format.

The sample text input used in this example is: `two men having a conversation.`

In [ ]:
import uuid
query_prefix = f'{s3_prefix}/input/{uuid.uuid4()}'

# Create an input embedding
response = bedrock.start_async_invoke(
    modelId=model_id,
    modelInput={
        "inputType": "text",
        "inputText": "two men having a conversation"
    },
    outputDataConfig={
        "s3OutputDataConfig": {
            "s3Uri": f's3://{s3_bucket}/{query_prefix}'
        }
    }
)

# Print Job ID
invocation_arn = response["invocationArn"]
print("Async Job Started")
print("Invocation Arn:", invocation_arn)

In [ ]:
query = wait_for_output_file(s3_bucket, query_prefix, invocation_arn)
display(JSON(query))

## Search the Vector Store
We now perform a similarity search against the vector index.
- The `indices` represent the positions of video clips within the original video embedding results.
- The `distances` indicate the similarity scores of these clips in the same order. A higher score means the clip is more similar to the search input.

In [ ]:
# Create query vector
query_vector = query[0]["embedding"] / np.linalg.norm(query[0]["embedding"])
query_vector = query_vector.reshape(1, -1)

# Perform search
k = 5  # number of nearest neighbors
distances, indices = index.search(query_vector, k)

# Show results
print("Nearest indices:", indices)
print("Similarity scores:", distances)


Now, we display the video to help you visualize the clips returned from the search.

In [ ]:
# Format data for display
start_times = []

counter = 0
for idx in indices[0]:
    item = output[idx]
    #print(idx, item["embeddingOption"], item["startSec"], item["endSec"])
    start_times.append((round(item["startSec"],2), f'{round(float(item["startSec"]),2)} - {round(float(item["endSec"]),2)}s (score: {round(float(distances[0][counter]),3)})'))
    counter += 1

In [ ]:
from IPython.display import HTML
import boto3

# Generate a presigned URL for the video in S3
s3 = boto3.client('s3')
url = s3.generate_presigned_url(
    ClientMethod='get_object',
    Params={'Bucket': s3_bucket, 'Key': s3_input_key},
    ExpiresIn=3600
)

Clicking the buttons below the video will take you to the timestamp where each clip begins.

In [ ]:
# Generate buttons HTML
buttons_html = ''.join([
    f'<button onclick="jumpTo({time})">{label}</button> '
    for time, label in start_times
])

html = f"""
<video id="videoPlayer" width="640" controls>
  <source src="{url}" type="video/mp4">
  Your browser does not support the video tag.
</video>

<div style="margin-top:10px;display:block;">
  {buttons_html}
</div>

<script>
  var video = document.getElementById('videoPlayer');

  function jumpTo(time) {{
    video.currentTime = time;
    video.play();
  }}
</script>
"""

display(HTML(html))

## Cleanup
Delete the video and the embedding files from S3

In [ ]:
# List all objects under the prefix
response = s3.list_objects_v2(Bucket=s3_bucket, Prefix=s3_prefix)

if 'Contents' in response:
    # Create a list of object identifiers to delete
    objects_to_delete = [{'Key': obj['Key']} for obj in response['Contents']]

    # Delete the objects
    s3.delete_objects(
        Bucket=s3_bucket,
        Delete={'Objects': objects_to_delete}
    )
    print(f"Deleted {len(objects_to_delete)} objects from '{s3_prefix}' in bucket '{s3_bucket}'.")
else:
    print(f"No objects found under prefix '{s3_prefix}'.")